In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.autograd import Variable
import argparse
import time

# input id
id_ = 1000

# setup training parameters
parser = argparse.ArgumentParser(description='PyTorch MNIST Training')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--test-batch-size', type=int, default=128, metavar='N',
                    help='input batch size for testing (default: 128)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')


args = parser.parse_args(args=[]) 

# judge cuda is available or not
use_cuda = not args.no_cuda and torch.cuda.is_available()
#device = torch.device("cuda" if use_cuda else "cpu")
device = torch.device("cpu")

torch.manual_seed(args.seed)
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

############################################################################
################################don't modify################################
############################################################################
class Dataset(Dataset):
    """User defined class to build a datset using Pytorch class Dataset."""
    
    def __init__(self, data, transform = None):
        """Method to initilaize variables.""" 
        self.data_ = list(data.values)
        self.transform = transform
        
        label = []
        image = []
        
        for i in self.data_:
             # first column is of labels.
            label.append(i[0])
            image.append(i[1:]/255)
        self.labels = np.asarray(label)
        # Dimension of Images = 28 * 28 * 1. where height = width = 28 and color_channels = 1.
        self.images = np.asarray(image).reshape(-1, 28, 28, 1).astype('float32')

    def __getitem__(self, index):
        label = self.labels[index]
        image = self.images[index]
        
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.images)

train_set = torchvision.datasets.FashionMNIST(root='../data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True)

# define fully connected network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        output = F.log_softmax(x, dim=1)
        return output
############################################################################
############################################################################
############################################################################

'generate adversarial data, you can define your adversarial method'
def adv_attack(model, X, y, device):
    X_adv = Variable(X.data)
    random_noise = torch.FloatTensor(*X_adv.shape).uniform_(-0.3, 0.3).to(device)
    X_adv = Variable(X_adv.data + random_noise)
    return X_adv

'train function, you can use adversarial training'
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.view(data.size(0),28*28)
        
        #use adverserial data to train the defense model
        #adv_data = adv_attack(model, data, target, device=device)
        
        #clear gradients
        optimizer.zero_grad()
        
        #compute loss
        #loss = F.cross_entropy(model(adv_data), target)
        loss = F.cross_entropy(model(data), target)
        
        #get gradients and update
        loss.backward()
        optimizer.step()

'predict function'
def eval_test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.view(data.size(0),28*28)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = correct / len(test_loader.dataset)
    return test_loss, test_accuracy

def eval_adv_test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            adv_data = adv_attack(model, data, target, device=device)
            adv_data = adv_data.view(adv_data.size(0),28*28)
            output = model(adv_data)
            test_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = correct / len(test_loader.dataset)
    return test_loss, test_accuracy

'main function, train the dataset and print train loss, test loss for each epoch'
def train_model():
    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr)
    for epoch in range(1, args.epochs + 1):
        start_time = time.time()
        
        #training
        train(args, model, device, train_loader, optimizer, epoch)
        
        #get trnloss and testloss
        trnloss, trnacc = eval_test(model, device, train_loader)
        advloss, advacc = eval_adv_test(model, device, train_loader)
        
        #print trnloss and testloss
        print('Epoch '+str(epoch)+': '+str(int(time.time()-start_time))+'s', end=', ')
        print('trn_loss: {:.4f}, trn_acc: {:.2f}%'.format(trnloss, 100. * trnacc), end=', ')
        print('adv_loss: {:.4f}, adv_acc: {:.2f}%'.format(advloss, 100. * advacc))
    
    #save the model
    torch.save(model.state_dict(), str(id_)+'.pt')
    return model

'compute perturbation distance'
def p_distance(model, train_loader, device):
    p = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        adv_data = adv_attack(model, data, target, device=device)
        p += torch.norm(data-adv_data, float('inf'))
    print('epsilon p: ',p/batch_idx)

'annotate the following code in the submission file'
#model = train_model()
#p_distance(model, train_loader, device)


/LOCAL2/gjin/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch 1: 11s, trn_loss: 1.8947, trn_acc: 37.22%, adv_loss: 1.8992, adv_acc: 37.17%
Epoch 2: 11s, trn_loss: 1.0717, trn_acc: 60.74%, adv_loss: 1.0803, adv_acc: 60.30%
Epoch 3: 11s, trn_loss: 0.8675, trn_acc: 65.08%, adv_loss: 0.8785, adv_acc: 65.17%
Epoch 4: 11s, trn_loss: 0.7730, trn_acc: 71.28%, adv_loss: 0.7858, adv_acc: 70.74%
Epoch 5: 10s, trn_loss: 0.6936, trn_acc: 74.45%, adv_loss: 0.7090, adv_acc: 73.87%
Epoch 6: 10s, trn_loss: 0.6499, trn_acc: 76.08%, adv_loss: 0.6695, adv_acc: 75.24%
Epoch 7: 10s, trn_loss: 0.6133, trn_acc: 78.57%, adv_loss: 0.6342, adv_acc: 77.63%
Epoch 8: 10s, trn_loss: 0.5689, trn_acc: 79.84%, adv_loss: 0.5938, adv_acc: 78.65%
Epoch 9: 10s, trn_loss: 0.5670, trn_acc: 79.51%, adv_loss: 0.5957, adv_acc: 78.31%
Epoch 10: 11s, trn_loss: 0.5311, trn_acc: 81.41%, adv_loss: 0.5626, adv_acc: 79.99%
epsilon p:  tensor(0.3006)
